In [7]:
import os
import json
import pathlib

inputpath = '/home/nps2dsoccer/logs/archive.robocup.info/Soccer/Simulation/2D/logs/'
outputpath = '/home/nps2dsoccer/logs/dataset/shoots/'

# count = 0
for log_file in pathlib.Path(inputpath).glob('**/*.rcg.gz'):
    # count = count + 1
    # if count < 4 :
            
    file_length = len(log_file.stem)
   
    com_filepath = str(log_file)
    # toll = len(com_filepath)
    print(1, ' Src: ', com_filepath[len(inputpath): len(com_filepath) - file_length - 3])
    structure = os.path.join(outputpath, com_filepath[len(inputpath): len(com_filepath) - file_length - 3 ]) #  3 is used to adjust the '.gz' of the file name that is not considered by stem
    print(2, ' Dataset: ', structure)
    print(3, ' File: ',log_file.stem, file_length)

    if not os.path.isdir(structure):
        os.makedirs(structure)
    gen_shoot_situations(log_file, structure)


1  Src:  RoboCup/2019/SeedingRound/GroupA/
2  Dataset:  /home/nps2dsoccer/logs/dataset/shoots/RoboCup/2019/SeedingRound/GroupA/
3  File:  20190704094758-Fractals2019_2-vs-MT2019_1.rcg 45
Goal Count:  1
Lenght of line_history:  15  Lenght of cycle_hostory:  3  Lenght of potent_cycle 3
Goal Count:  2
Lenght of line_history:  15  Lenght of cycle_hostory:  3  Lenght of potent_cycle 3
Goal Count:  3
Lenght of line_history:  15  Lenght of cycle_hostory:  3  Lenght of potent_cycle 3
1  Src:  RoboCup/2019/SeedingRound/GroupA/
2  Dataset:  /home/nps2dsoccer/logs/dataset/shoots/RoboCup/2019/SeedingRound/GroupA/
3  File:  20190704103528-Receptivity_0-vs-Fractals2019_0.rcg 50
1  Src:  RoboCup/2019/SeedingRound/GroupA/
2  Dataset:  /home/nps2dsoccer/logs/dataset/shoots/RoboCup/2019/SeedingRound/GroupA/
3  File:  20190704102356-ITAndroids_0-vs-MT2019_3.rcg 43
Goal Count:  1
Lenght of line_history:  15  Lenght of cycle_hostory:  3  Lenght of potent_cycle 3
Goal Count:  2
Lenght of line_history:  15  

In [4]:
from collections import deque

def gen_shoot_situations(src, dest): 
    line_history = deque(maxlen=15)  # Decided based on matched of 2019 Elimination Round (Max 15 cycle)
    cycle_history = deque(maxlen=3)  # Safely allocated to read a situation after discussion with Thiago
    potent_cycle = deque(maxlen=3)
    count = 0
    with open(src) as inputs:
        for line in inputs:
            if " goal_l" in line : 
                count += 1
                print('Goal Count: ' , count)
                cycle_slice = find_shoot(line_history, cycle_history, potent_cycle )
                # print(cycle_slice)
                
                # print(*line_history, sep='\n')

                # for cycle in line_history:
                #     if " 0x3 " in cycle  : # check if kick is found any cycle prior to the goal 
                #         cycle_history.append(cycle)
                #         potent_cycle = cycle_history.copy()
                        
                #     else:
                #         # to keep just the two cycle prior where the kick is found
                #         cycle_history.append(cycle)
                
                # # print(cycle_history.popleft())  # pop the last two clcyle prior where the kick is found
                # cycle_slice = potent_cycle.popleft()
                # print(cycle_slice)
                # print( line, 'L')

                gen_json(count, cycle_slice , dest, 'L')

                
                # Clear history so if two goal_l/goal_r seen in close proximity, we don't
                # echo some lines twice
                # line_history.clear()
            elif " goal_r" in line:
                count += 1
                print('Goal Count: ' , count)
                cycle_slice = find_shoot(line_history, cycle_history, potent_cycle )
                # print(cycle_slice)
                gen_json(count, cycle_slice , dest, 'R')

                # print(cycle_slice)
                # print( line, 'R')

            else:
                # When deque reaches 15 lines, will automatically evict oldest
                line_history.append(line)
                

In [5]:
def find_shoot(line_history, cycle_history, potent_cycle):
    for cycle in line_history:
        if is_valid_cycle(cycle): # Check if cycle is valid
            
            # When the state of the player is overwritten by other states (say yellow; ), returen the last last line of history as a default
            cycle_slice = cycle

            if " 0x3 " in cycle  : # check if kick is found any cycle prior to the goal 
                cycle_history.append(cycle)
                potent_cycle = cycle_history.copy()
                
            else:
                # to keep just the two cycle prior where the kick is found
                cycle_history.append(cycle)

    # print(cycle_history.popleft())  
    print('Lenght of line_history: ',len(line_history), ' Lenght of cycle_hostory: ', len(cycle_history), ' Lenght of potent_cycle', len(potent_cycle) )
    if potent_cycle: 
        # When kick is found based of state of the player (0x3)
        cycle_slice = potent_cycle.popleft() # pop the two clcyle prior where the kick is found (two cycle a buffer)

    # print(cycle_slice)
    
    line_history.clear()
    return cycle_slice


In [1]:
def is_valid_cycle(cycle):
    # is_valid = False
    # if len(cucle) > 100:
    #     is_valie = True
    return True if len(cycle) > 100 else False


In [6]:
def create_cycleList(cycleSlice):
    # create List Of List containing values of ball and each player
    cycle = list()
    cycle_list = cycleSlice.split("))")
    del cycle_list[-1]
    for i in cycle_list:
        cycle.append(i + "))")

    cyclenew = list()
    cyclenew.insert(0,cycle[0].split()[:7])
    cyclenew.append(cycle[0].split()[7:])
    del cycle[0]

    for i in cycle:
        cyclenew.append(i.split())

    return cyclenew


# Functions to extract values of each player and ball from log cycle

def ball_data(data):
    for i in range(len(data)):
        if(data[i]=="((b)"):
            ball_x = float(data[i+1])
            ball_y = float(data[i+2])
            ballvel_x = float(data[i+3])
            ballvel_y = float(data[i+4][:-1])
    return ball_x, ball_y, ballvel_x, ballvel_y

def player_position(data, side_pattern):
    for i in range(len(data)):
        if(data[i]== side_pattern):
            unum = int(data[i+1][:-1])
            pos_x = float(data[i+4])
            pos_y = float(data[i+5])
            vel_x = float(data[i+6])
            vel_y = float(data[i+7])
            b_angle = float(data[i+8])
            h_angle = float(data[i+9])
        else:
            continue
    return unum, pos_x, pos_y, vel_x, vel_y, b_angle, h_angle

def player_view(data, pattern):
    for i in range(len(data)):
        if(data[i]==pattern):
            view_width = int(data[i+2][:-1])
        else:
            continue
    return view_width

def player_power(data, pattern):
    if pattern in data:
        for i in range(len(data)):
            if(data[i]== pattern):
                stamina = float(data[i+1])
                effort = float(data[i+2])
                recovery = float(data[i+3])
                turnneck = float(data[i+4][:-1])
            else:
                continue
    else:
        stamina = 0
        effort = 0
        recovery = 0
        turnneck = 0
    return stamina, effort, recovery, turnneck

def player_face(data, pattern):
    if pattern in data:
        for i in range(len(data)):
            if(data[i]== pattern):
                playerside = data[i+1]
                playerunum = int(data[i+2][:-1])
            else:
                continue
    else:
        playerside = 'na'
        playerunum = -1
    return playerside, playerunum

def counts(data, pattern):
    if pattern in data:
        for i in range(len(data)):
            if(data[i]==pattern):
                K_count = int(data[i+1])
                d_count = int(data[i+2])
                t_count = int(data[i+3])
                say_count = int(data[i+4])
                turnNeck_count = int(data[i+5])
                catch_count = int(data[i+6])
                m_count = int(data[i+7])
                change_view = int(data[i+8])
                point_to = int(data[i+9])
                att_to = int(data[i+10])
                tackle_count = int(data[i+11][:-2])
            else:
                continue
    else:
        K_count = d_count = t_count = say_count = turnNeck_count = catch_count = m_count = change_view = point_to = att_to = tackle_count = 0  
    return K_count, d_count, t_count, say_count, turnNeck_count, catch_count, m_count, change_view, point_to, att_to, tackle_count



def gen_json(count, cycle_slice, dest, side):

    cycle_dataset = create_cycleList(cycle_slice)

    player = []
    ball_x, ball_y, ballvel_x, ballvel_y = ball_data(cycle_dataset[0])
    ball = dict([("posx",ball_x),("posy",ball_y),("velx",ballvel_x),("vely",ballvel_y)])   #create a dictionary of ball postion and velocity


    for i in range(1,len(cycle_dataset)):           #create the player dictionary of dictionary that will store data about each players
        if(i<12):
            unum, pos_x, pos_y, vel_x, vel_y, b_angle, h_angle = player_position(cycle_dataset[i],"((l")
            viewWidth = player_view(cycle_dataset[i],"(v")
            stamina, effort, recovery, turn_neck = player_power(cycle_dataset[i], "(s")
            playerside, playerunum = player_face(cycle_dataset[i],"(f")
            K_count, d_count, t_count, say_count, turnNeck_count, catch_count, m_count, change_view, point_to, att_to, tackle_count = counts(cycle_dataset[i],"(c")
        
            player_dict = dict([("unum",unum),("side","l"),("posx",pos_x),("posy",pos_y),("velx",vel_x),("vely",vel_y), ("bangle",b_angle), ("hangle",h_angle), ("viewWidth",viewWidth), ("stamina",stamina), ("effort",effort),("recovery",recovery), ("turnNeck", turn_neck), ("facePlayerSide", playerside), ("facePlayerNum",playerunum), ("kickCount",K_count), ("dashCount",d_count),("turnCount",t_count),("sayCount",say_count), ("turnNeckCount", turnNeck_count), ("catchCount",catch_count), ("moveCount",m_count), ("changeView",change_view), ("pointtoCount",point_to), ("attentoCount",att_to), ("tackleCount",tackle_count)])
            player.append(player_dict)
        elif(i>11):
            unum, pos_x, pos_y, vel_x, vel_y, b_angle, h_angle = player_position(cycle_dataset[i],"((r")
            viewWidth = player_view(cycle_dataset[i],"(v")
            stamina, effort, recovery, turn_neck = player_power(cycle_dataset[i], "(s")
            playerside, playerunum = player_face(cycle_dataset[i], '(f')
            K_count, d_count, t_count, say_count, turnNeck_count, catch_count, m_count, change_view, point_to, att_to, tackle_count = counts(cycle_dataset[i],"(c")
        
            player_dict = dict([("unum",unum),("side","r"),("posx",pos_x),("posy",pos_y),("velx",vel_x),("vely",vel_y), ("bangle",b_angle), ("hangle",h_angle), ("viewWidth",viewWidth), ("stamina",stamina), ("effort",effort),("recovery",recovery), ("turnNeck", turn_neck), ("facePlayerSide", playerside), ("facePlayerNum",playerunum), ("kickCount",K_count), ("dashCount",d_count),("turnCount",t_count),("sayCount",say_count), ("turnNeckCount", turnNeck_count), ("catchCount",catch_count), ("moveCount",m_count), ("changeView",change_view), ("pointtoCount",point_to), ("attentoCount",att_to), ("tackleCount",tackle_count)])
            player.append(player_dict)

    final_dict={"Side" : side,"ball":ball,"player":{"plyer": player}}

    # # Dump Dictionary to JSON file

    with open(f'{dest}'+f'shootsituation-{count}.json', 'w') as fp:
        json.dump(final_dict, fp)
    # return final_di

In [9]:
out = dest = os.getcwd()
pa = os.path.join(out, 'a', 'b')
print(pa)

NameError: name 'path' is not defined